In [10]:
from __future__ import print_function
import os, sys
import logging
import six, csv
import numpy as np
import pandas as pd
import glob
import tqdm

# import matplotlib
# matplotlib.use("AGG")
import matplotlib.pyplot as plt

import pydicom as dicom
import nibabel as nib

import SimpleITK as sitk

import radiomics
from radiomics import featureextractor
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm

In [11]:
train_image_folder = "train-images/image/"
train_list = os.listdir(train_image_folder)[:]

bad_train = ['.DS_Store']
for s in bad_train:
    if (s in train_list):
        train_list.remove(s)

In [12]:
def load_nii_gz(label_fpath, transpose=False):
    data = nib.load(label_fpath)
    pixel_array = data.get_fdata()
    if transpose:
        pixel_array = np.transpose(pixel_array, axes=(2,1,0))
    return pixel_array 

In [13]:
map_image_t1_list = sorted(glob.glob(os.path.join(train_image_folder, '*/t1.nii.gz')))
map_image_t1ce_list = sorted(glob.glob(os.path.join(train_image_folder, '*/t1ce.nii.gz')))
map_image_t2_list = sorted(glob.glob(os.path.join(train_image_folder, '*/t2.nii.gz')))
map_image_flair_list = sorted(glob.glob(os.path.join(train_image_folder, '*/flair.nii.gz')))
map_image_seg_list = sorted(glob.glob(os.path.join(train_image_folder, '*/seg.nii.gz')))

In [14]:
df_age = pd.read_csv("train-survival.csv", )

In [15]:
df = df_age.sort_values(by=["Encode"])
df = df.drop(columns=["Unnamed: 0"])
df.reset_index(drop=True, inplace=True)
df

,Encode,Age,Survival
0,0j3l3scl9r2iravi9f5o0t9azjp7zb8n,56.419,1155
1,0xjh8vt6t6e3ewa7t53eh0ejfghx8k88,45.244,614
2,2l9tka5g9sbcdjn6a3rkttwcovmz8e02,59.693,698
3,3m4xjm6tkdadi1wyjjuncdituz0kzq7l,58.258,439
4,5v40qp8mri7rjg4fzwcggr4216g42axb,55.532,180
5,6jnw70prdhuojd76urja8tkwa4f2bfaq,52.263,616
6,80a9j2dc3jeffhks53usnx3ibs0j03mm,46.551,375
7,8r467ry7c4ck5wyt269tokv7e18gi4tr,68.726,152
8,9b6hjbfct0jqydmfg4n0ggsvz0td08rt,77.337,522
9,9e88ooy25utxmh6vdr4ut4st1h1fly7q,68.020,120


In [16]:
map_df =pd.DataFrame(data={"Encode": df['Encode'], "t1": map_image_t1_list, "t1ce": map_image_t1ce_list,
                          "t2": map_image_t2_list, "flair": map_image_flair_list, "seg": map_image_seg_list,
                          "Age": df['Age'], "Survival": df['Survival']})

In [17]:
map_df.head()

,Encode,t1,t1ce,t2,flair,seg,Age,Survival
0,0j3l3scl9r2iravi9f5o0t9azjp7zb8n,train-images/image/0j3l3scl9r2iravi9f5o0t9azjp...,train-images/image/0j3l3scl9r2iravi9f5o0t9azjp...,train-images/image/0j3l3scl9r2iravi9f5o0t9azjp...,train-images/image/0j3l3scl9r2iravi9f5o0t9azjp...,train-images/image/0j3l3scl9r2iravi9f5o0t9azjp...,56.419,1155
1,0xjh8vt6t6e3ewa7t53eh0ejfghx8k88,train-images/image/0xjh8vt6t6e3ewa7t53eh0ejfgh...,train-images/image/0xjh8vt6t6e3ewa7t53eh0ejfgh...,train-images/image/0xjh8vt6t6e3ewa7t53eh0ejfgh...,train-images/image/0xjh8vt6t6e3ewa7t53eh0ejfgh...,train-images/image/0xjh8vt6t6e3ewa7t53eh0ejfgh...,45.244,614
2,2l9tka5g9sbcdjn6a3rkttwcovmz8e02,train-images/image/2l9tka5g9sbcdjn6a3rkttwcovm...,train-images/image/2l9tka5g9sbcdjn6a3rkttwcovm...,train-images/image/2l9tka5g9sbcdjn6a3rkttwcovm...,train-images/image/2l9tka5g9sbcdjn6a3rkttwcovm...,train-images/image/2l9tka5g9sbcdjn6a3rkttwcovm...,59.693,698
3,3m4xjm6tkdadi1wyjjuncdituz0kzq7l,train-images/image/3m4xjm6tkdadi1wyjjuncdituz0...,train-images/image/3m4xjm6tkdadi1wyjjuncdituz0...,train-images/image/3m4xjm6tkdadi1wyjjuncdituz0...,train-images/image/3m4xjm6tkdadi1wyjjuncdituz0...,train-images/image/3m4xjm6tkdadi1wyjjuncdituz0...,58.258,439
4,5v40qp8mri7rjg4fzwcggr4216g42axb,train-images/image/5v40qp8mri7rjg4fzwcggr4216g...,train-images/image/5v40qp8mri7rjg4fzwcggr4216g...,train-images/image/5v40qp8mri7rjg4fzwcggr4216g...,train-images/image/5v40qp8mri7rjg4fzwcggr4216g...,train-images/image/5v40qp8mri7rjg4fzwcggr4216g...,55.532,180


In [18]:
map_df.shape

(39, 8)

In [19]:
X_age = np.array(map_df['Age'])
y = map_df['Survival']
print(X_age.shape, y.shape)

(39,) (39,)


In [41]:
np.save("y_survival.npy", y)

In [20]:
X_t1 = np.zeros((map_df.shape[0], 155, 240, 240))
X_t1ce = np.zeros((map_df.shape[0], 155, 240, 240))
X_t2 = np.zeros((map_df.shape[0], 155, 240, 240))
X_flair = np.zeros((map_df.shape[0], 155, 240, 240))
X_seg = np.zeros((map_df.shape[0], 155, 240, 240))
for i in range(map_df.shape[0]):
    X_t1[i, :, :, :] = load_nii_gz(map_df.iloc[i]['t1'], transpose=True)
    X_t1ce[i, :, :, :] = load_nii_gz(map_df.iloc[i]['t1ce'], transpose=True)
    X_t2[i, :, :, :] = load_nii_gz(map_df.iloc[i]['t2'], transpose=True)
    X_flair[i, :, :, :] = load_nii_gz(map_df.iloc[i]['flair'], transpose=True)
    X_seg[i, :, :, :] = load_nii_gz(map_df.iloc[i]['seg'], transpose=True)

In [21]:
X_t1 = X_t1.astype(np.int16)
X_t1ce = X_t1ce.astype(np.int16)
X_t2 = X_t2.astype(np.int16)
X_flair = X_flair.astype(np.int16)
X_seg = X_seg.astype(np.int16)

In [28]:
np.savez_compressed("X_image.npz", t1=X_t1, t1ce=X_t1ce, t2=X_t2, flair=X_flair, seg=X_seg, age=X_age)